# Building RESTful APIs with FastAPI

FastAPI is a modern web framework for Python that makes it easy to build robust and high-performance APIs. It is designed with developer productivity in mind, offering features like automatic validation, interactive documentation, and asynchronous programming.

This tutorial will guide you through building a simple RESTful API using FastAPI.

---

## Prerequisites

Before starting, ensure you have the following:

- Python 3.7 or higher installed on your machine
- Basic knowledge of Python programming
- A text editor or an integrated development environment (IDE) like VS Code or PyCharm

---

## Step 1: Setting Up Your Environment

### 1.1 Install FastAPI and Uvicorn

FastAPI is the framework we'll use to build the API, and Uvicorn is an ASGI server to run the application.

```bash
pip install fastapi uvicorn
```

### 1.2 Create a New Project

Organize your files in a directory structure like this:

```
fastapi_tutorial/
    main.py
```

---

## Step 2: Creating Your First Endpoint

Open `main.py` and add the following code:

```python
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"message": "Welcome to the FastAPI tutorial!"}
```

### Explanation

- `FastAPI()`: Creates a FastAPI application instance.
- `@app.get("/")`: Defines a GET endpoint at the root path (`/`).
- `read_root`: A function that handles requests to the root path and returns a JSON response.

---

## Step 3: Running the Application

Start the application using Uvicorn:

```bash
uvicorn main:app --reload
```

- Open a browser and navigate to `http://127.0.0.1:8000` to see the response.
- Access the interactive API documentation at `http://127.0.0.1:8000/docs`.

---

## Step 4: Adding More Endpoints

Let’s expand the API to manage a collection of items (e.g., books).

### 4.1 Define the Data Model

```python
from pydantic import BaseModel

class Book(BaseModel):
    title: str
    author: str
    year: int
```

### 4.2 Create Endpoints

Update `main.py`:

```python
books = []

@app.get("/books")
def get_books():
    return books

@app.post("/books")
def add_book(book: Book):
    books.append(book.dict())
    return {"message": "Book added successfully!"}

@app.get("/books/{book_id}")
def get_book(book_id: int):
    if book_id < len(books):
        return books[book_id]
    return {"error": "Book not found"}
```

### Explanation

- `BaseModel`: A Pydantic model used for data validation.
- `@app.get("/books")`: Retrieves the list of books.
- `@app.post("/books")`: Adds a new book to the collection.
- `@app.get("/books/{book_id}")`: Fetches a book by its index in the list.

---

## Step 5: Improving Validation

You can add validation rules directly in the `Book` model:

```python
class Book(BaseModel):
    title: str
    author: str
    year: int

    @validator("year")
    def check_year(cls, value):
        if value > 2024:
            raise ValueError("Year must be 2024 or earlier.")
        return value
```

---

## Step 6: Making the API Asynchronous

FastAPI natively supports asynchronous programming with `async def`.

```python
@app.get("/books")
async def get_books():
    return books

@app.post("/books")
async def add_book(book: Book):
    books.append(book.dict())
    return {"message": "Book added successfully!"}
```

---

## Step 7: Adding Middleware and Dependencies

### 7.1 Middleware

Middleware allows you to execute code before or after requests.

```python
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)
```

### 7.2 Dependencies

Dependencies can be used to inject reusable logic into endpoints.

```python
from fastapi import Depends

def common_parameters(q: str = None):
    return {"query": q}

@app.get("/search")
def search(params: dict = Depends(common_parameters)):
    return params
```

---

## Step 8: Testing the API

Use FastAPI’s built-in test client to write unit tests.

```python
from fastapi.testclient import TestClient

client = TestClient(app)

def test_read_root():
    response = client.get("/")
    assert response.status_code == 200
    assert response.json() == {"message": "Welcome to the FastAPI tutorial!"}
```

---

## Step 9: Deployment

### 9.1 Production Deployment

Use a production-ready server like Gunicorn with Uvicorn workers:

```bash
gunicorn -w 4 -k uvicorn.workers.UvicornWorker main:app
```

### 9.2 Dockerizing the Application

Create a `Dockerfile`:

```dockerfile
FROM python:3.9-slim
WORKDIR /app
COPY . /app
RUN pip install fastapi uvicorn
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "80"]
```

Build and run the container:

```bash
docker build -t fastapi-app .
docker run -d -p 80:80 fastapi-app
```

---

## Conclusion

FastAPI simplifies building RESTful APIs with features like data validation, interactive documentation, and asynchronous support. By following this tutorial, you’ve learned how to create a basic API, add endpoints, validate data, and deploy the application. Explore more advanced features like WebSockets, authentication, and database integrations to expand your skills.

